In [89]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Read and Preprocess Data

In [90]:
baseball = pd.read_csv('data/baseball.csv')

baseball = baseball.drop(['Name', 'Age', 'Name-additional'], axis = 1)
baseball['Salary'] = baseball['Salary'].str.replace('$', '').astype(float)

baseball['C'] = baseball['Position'].apply(lambda x: 1 if 'C' in x else 0)
baseball['1B'] = baseball['Position'].apply(lambda x: 1 if '1B' in x else 0)
baseball['2B'] = baseball['Position'].apply(lambda x: 1 if '2B' in x else 0)
baseball['3B'] = baseball['Position'].apply(lambda x: 1 if '3B' in x else 0)
baseball['SS'] = baseball['Position'].apply(lambda x: 1 if 'SS' in x else 0)
baseball['OF'] = baseball['Position'].apply(lambda x: 1 if 'OF' in x else 0)

baseball['Num_Pos'] = baseball[['C', '1B', '2B', '3B', 'SS', 'OF']].sum(axis = 1)
baseball = baseball.drop(['Position'], axis = 1)

In [91]:
X = baseball.drop(['Salary'], axis = 1)
y = baseball['Salary']
#y = (baseball['Salary'] - np.mean(baseball['Salary'])) / np.std(baseball['Salary'])

cat_columns = ['Tm', 'Lg', 'Acquired', 'Bat']
num_columns = [col for col in X.columns if col not in cat_columns + ['C', '1B', '2B', '3B', 'SS', 'OF']]

In [62]:
#num_columns = ['Def-Inn', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rdrs', 'RAA', 'WAA', 'RAR',
#               'WAR', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP',
#               'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'Num_Pos']
#cat_columns = ['Tm', 'Acquired']

#X = X[num_columns + cat_columns]

In [92]:
cat_transformer = Pipeline(
    steps = [
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

num_transformer = Pipeline(
    steps = [
        ('scale', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, num_columns),
        ('cat', cat_transformer, cat_columns)
    ], remainder = 'passthrough'
)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 621)

## Create and Fit Pipeline Random Forest

In [94]:
pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators = 150, min_samples_leaf = 10))
    ]
)

In [95]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cont',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler())]),
                                                  ['Def-Inn', 'PO', 'A', 'E',
                                                   'DP', 'Fld%', 'Rdrs',
                                                   'Season', 'RAA', 'WAA',
                                                   'RAR', 'WAR', 'PA', 'AB',
                                                   'R', 'H', 'HR', 'RBI', 'SB',
                                                   'CS', 'BB', 'SO', 'BA',
                                                   'OBP', 'SLG', 'OPS', 'OPS+',
                                                   'TB', 'GDP', 'HBP', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Tm', 'Lg', 'Acquired',
                                                   'Bat'])])),
                ('model',
                 RandomForestRegressor(min_samples_leaf=10, n_estimators=150))])

In [96]:
train_mse = mean_squared_error(y_train, pipe.predict(X_train))
test_mse = mean_squared_error(y_test, pipe.predict(X_test))

In [97]:
print(f'Train MSE: {np.sqrt(train_mse)}')
print(f'Test MSE: {np.sqrt(test_mse)}')
print(f'Mean of Y: {np.std(y)}')

Train MSE: 3990549.7988496535
Test MSE: 5360333.8405447425
Mean of Y: 6395365.8881033715


## Create and Fit Pipeline XGBoost

In [98]:
from xgboost import XGBRegressor

In [99]:
pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', XGBRegressor())
    ]
)

In [100]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cont',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler())]),
                                                  ['Def-Inn', 'PO', 'A', 'E',
                                                   'DP', 'Fld%', 'Rdrs',
                                                   'Season', 'RAA', 'WAA',
                                                   'RAR', 'WAR', 'PA', 'AB',
                                                   'R', 'H', 'HR', 'RBI', 'SB',
                                                   'CS', 'BB', 'SO', 'BA',
                                                   'OBP', 'SLG', 'OPS', 'OPS+',
                                                   'TB', 'GDP', 'HBP', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [101]:
train_mse = mean_squared_error(y_train, pipe.predict(X_train))
test_mse = mean_squared_error(y_test, pipe.predict(X_test))

In [102]:
print(f'Train MSE: {np.sqrt(train_mse)}')
print(f'Test MSE: {np.sqrt(test_mse)}')
print(f'Mean of Y: {np.std(y)}')

Train MSE: 996000.2897536749
Test MSE: 5174242.088274057
Mean of Y: 6395365.8881033715


## Create and Fit CNN

In [108]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler

In [105]:
baseball = pd.read_csv('data/baseball.csv')

baseball = baseball.drop(['Name', 'Age', 'Name-additional'], axis = 1)
baseball['Salary'] = baseball['Salary'].str.replace('$', '').astype(float)

baseball['C'] = baseball['Position'].apply(lambda x: 1 if 'C' in x else 0)
baseball['1B'] = baseball['Position'].apply(lambda x: 1 if '1B' in x else 0)
baseball['2B'] = baseball['Position'].apply(lambda x: 1 if '2B' in x else 0)
baseball['3B'] = baseball['Position'].apply(lambda x: 1 if '3B' in x else 0)
baseball['SS'] = baseball['Position'].apply(lambda x: 1 if 'SS' in x else 0)
baseball['OF'] = baseball['Position'].apply(lambda x: 1 if 'OF' in x else 0)

baseball['Num_Pos'] = baseball[['C', '1B', '2B', '3B', 'SS', 'OF']].sum(axis = 1)
baseball = baseball.drop(['Position'], axis = 1)

cat_columns = ['Tm', 'Lg', 'Acquired', 'Bat']
num_columns = [col for col in X.columns if col not in cat_columns + ['C', '1B', '2B', '3B', 'SS', 'OF']]

baseball = pd.get_dummies(baseball, columns = cat_columns)
#baseball.info()

In [109]:
y = baseball['Salary']
X = baseball.drop(['Salary'], axis = 1)

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 621)

min_max_scalar = MinMaxScaler()
X_train = min_max_scalar.fit_transform(X_train)
X_test = min_max_scalar.transform(X_test)

In [114]:
model = Sequential()
model.add(Dense(1000, input_shape = (X_train.shape[1], ), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'linear'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1000)              88000     
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_5 (Dense)             (None, 500)               500500    
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_6 (Dense)             (None, 250)               125250    
                                                                 
 dropout_2 (Dropout)         (None, 250)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                

In [115]:
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

In [116]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)

In [118]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), callbacks = [es], epochs = 5000, batch_size = 50, verbose = 1)

Epoch 1/5000
81/81 [==============================] - 3s 21ms/step - loss: 65458166300672.0000 - mae: 4985864.5000 - val_loss: 65352918630400.0000 - val_mae: 4849592.0000
Epoch 2/5000
81/81 [==============================] - 1s 17ms/step - loss: 52406767321088.0000 - mae: 4311260.0000 - val_loss: 39986158108672.0000 - val_mae: 4401210.5000
Epoch 3/5000
81/81 [==============================] - 1s 17ms/step - loss: 37548873220096.0000 - mae: 4506814.5000 - val_loss: 38112038223872.0000 - val_mae: 4615650.0000
Epoch 4/5000
81/81 [==============================] - 1s 17ms/step - loss: 36517028298752.0000 - mae: 4466976.5000 - val_loss: 37347563405312.0000 - val_mae: 4535641.5000
Epoch 5/5000
81/81 [==============================] - 2s 22ms/step - loss: 35711050842112.0000 - mae: 4419239.0000 - val_loss: 36618111025152.0000 - val_mae: 4538241.5000
Epoch 6/5000
81/81 [==============================] - 1s 18ms/step - loss: 35180223922176.0000 - mae: 4383656.5000 - val_loss: 35963971567616.000

In [121]:
np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

32/32 [==============================] - 0s 4ms/step


4483556.670354802